<a href="https://colab.research.google.com/github/Gayathri-achari/Projects/blob/Stock-price-Prediction/Stock_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import GRU, Conv1D, Dense, Flatten, Input, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load stock data
data = pd.read_csv('/content/Google_test_data.csv')  # Ensure you have Google stock data
prices = data['Close'].values
prices = (prices - np.min(prices)) / (np.max(prices) - np.min(prices))  # Normalize

# Prepare dataset
seq_len = 7
X, y = [], []
for i in range(len(prices) - seq_len):
    X.append(prices[i:i+seq_len])
    y.append(prices[i+seq_len])
X, y = np.array(X), np.array(y)

def build_generator():
    inp = Input(shape=(seq_len, 1))
    x = GRU(64, return_sequences=True)(inp)
    x = GRU(64)(x)
    out = Dense(seq_len, activation='relu')(x)  # Ensure positive values
    # Reshape output to (None, seq_len, 1) which is the expected input shape for Conv1D in discriminator
    out = Reshape((seq_len, 1))(out)
    model = Model(inp, out)
    return model


# Discriminator model
def build_discriminator():
    inp = Input(shape=(seq_len, 1))
    x = Conv1D(64, kernel_size=3, activation='relu')(inp)
    x = Flatten()(x)
    out = Dense(1, activation='sigmoid')(x)
    model = Model(inp, out)
    return model

# Instantiate models
generator = build_generator()
discriminator = build_discriminator()

# Compile discriminator
# The discriminator will be trained independently on real and fake data.
discriminator.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')

# GAN model
# In the GAN, the generator's output is fed into the discriminator.
gan_input = Input(shape=(seq_len, 1))
generated_price_for_gan = generator(gan_input)
discriminator.trainable = False # Freeze discriminator weights when training the generator through the GAN
validity_of_generated = discriminator(generated_price_for_gan)
gan = Model(gan_input, validity_of_generated)
gan.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')

# Training
epochs = 15
batch_size = 32
for epoch in range(epochs):
    # Select a random batch of real data
    idx = np.random.randint(0, X.shape[0], batch_size)
    real_data = X[idx] # Shape is (batch_size, seq_len)

    # Ensure real_data has the correct shape (batch_size, seq_len, 1) for the models
    # We need to add the channel dimension for both real and fake data.
    real_data = real_data.reshape(batch_size, seq_len, 1)

    # Generate fake data from the generator
    # generator.predict expects input shape (batch_size, seq_len, 1)
    # The output of generator.predict will have shape (batch_size, seq_len, 1) due to the Reshape layer
    fake_data = generator.predict(real_data)

    # Labels for the discriminator
    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))

    # Train the discriminator
    # discriminator.train_on_batch expects input shape (batch_size, seq_len, 1)
    d_loss_real = discriminator.train_on_batch(real_data, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_data, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the generator through the GAN
    # We want the generator to fool the discriminator, so we use real_labels
    # gan.train_on_batch expects input shape (batch_size, seq_len, 1)
    g_loss = gan.train_on_batch(real_data, real_labels)

    print(f"Epoch {epoch+1}/{epochs} | D Loss: {d_loss:.4f} | G Loss: {g_loss:.4f}")

# Predict next-day stock price
# Ensure the input to predict is also (1, seq_len, 1)
# X[-1] has shape (seq_len,). Reshape to (1, seq_len, 1)
input_for_prediction = X[-1].reshape(1, seq_len, 1)
next_day_prediction = generator.predict(input_for_prediction)

# The output shape of generator.predict is (1, seq_len, 1).
# Depending on what you want to predict (e.g., the next single value or the next sequence),
# you'll need to adjust how you extract the result.
# If predicting the next single price based on the last sequence:
# The generator output is a sequence of 'seq_len' values. If the goal is to predict the single next value,
# the generator architecture might need adjustment (e.g., a Dense(1) layer at the end if not predicting a sequence).
# Assuming the intention was to predict the *next sequence* of length seq_len starting from the last data point:
print(f"Predicted Next-Day Prices Sequence: {next_day_prediction[0]}")
# If you truly meant the single next day price, you might need to reconsider the generator's output layer.
# For this code as written, it predicts a sequence of length seq_len.

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 1/15 | D Loss: 0.6930 | G Loss: 0.6923
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Epoch 2/15 | D Loss: 0.6931 | G Loss: 0.6921
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Epoch 3/15 | D Loss: 0.6934 | G Loss: 0.6920
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Epoch 4/15 | D Loss: 0.6934 | G Loss: 0.6918
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Epoch 5/15 | D Loss: 0.6934 | G Loss: 0.6917
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Epoch 6/15 | D Loss: 0.6936 | G Loss: 0.6915
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Epoch 7/15 | D Loss: 0.6937 | G Loss: 0.6913
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Epoch 8/15 | D Loss: 0.6938 | G Loss: 0.6911
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Epoch 9/15 | D Loss: 0.6939 | G Loss: 0.6909
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Epoch 10/15 | D Loss: 0.6941 | G Loss: 0.6907
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Epoch 11/15 | D Loss: 0.6942 | G Loss: 0.6904
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Epoch 12/15 | D Loss: 0.6943 | G Loss: 0.6902
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
E

In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import GRU, Conv1D, Dense, Flatten, Input, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load stock data
data = pd.read_csv('/content/Google_test_data.csv')  # Ensure you have Google stock data
prices = data['Close'].values
prices = (prices - np.min(prices)) / (np.max(prices) - np.min(prices))  # Normalize

# Prepare dataset
seq_len = 7
X, y = [], []
for i in range(len(prices) - seq_len):
    X.append(prices[i:i+seq_len])
    y.append(prices[i+seq_len])
X, y = np.array(X), np.array(y)

def build_generator():
    inp = Input(shape=(seq_len, 1))
    x = GRU(64, return_sequences=True)(inp)
    x = GRU(64)(x)
    out = Dense(seq_len, activation='relu')(x)  # Ensure positive values
    # Reshape output to (None, seq_len, 1) which is the expected input shape for Conv1D in discriminator
    out = Reshape((seq_len, 1))(out)
    model = Model(inp, out)
    return model


# Discriminator model
def build_discriminator():
    inp = Input(shape=(seq_len, 1))
    x = Conv1D(64, kernel_size=3, activation='relu')(inp)
    x = Flatten()(x)
    out = Dense(1, activation='sigmoid')(x)
    model = Model(inp, out)
    return model

# Instantiate models
generator = build_generator()
discriminator = build_discriminator()

# Compile discriminator
# The discriminator will be trained independently on real and fake data.
discriminator.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')

# GAN model
# In the GAN, the generator's output is fed into the discriminator.
gan_input = Input(shape=(seq_len, 1))
generated_price_for_gan = generator(gan_input)
discriminator.trainable = False # Freeze discriminator weights when training the generator through the GAN
validity_of_generated = discriminator(generated_price_for_gan)
gan = Model(gan_input, validity_of_generated)
gan.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')

# Training
epochs = 15
batch_size = 32
for epoch in range(epochs):
    # Select a random batch of real data
    idx = np.random.randint(0, X.shape[0], batch_size)
    real_data = X[idx] # Shape is (batch_size, seq_len)

    # Ensure real_data has the correct shape (batch_size, seq_len, 1) for the models
    # We need to add the channel dimension for both real and fake data.
    real_data = real_data.reshape(batch_size, seq_len, 1)

    # Generate fake data from the generator
    # generator.predict expects input shape (batch_size, seq_len, 1)
    # The output of generator.predict will have shape (batch_size, seq_len, 1) due to the Reshape layer
    fake_data = generator.predict(real_data)

    # Labels for the discriminator
    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))

    # Train the discriminator
    # discriminator.train_on_batch expects input shape (batch_size, seq_len, 1)
    d_loss_real = discriminator.train_on_batch(real_data, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_data, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the generator through the GAN
    # We want the generator to fool the discriminator, so we use real_labels
    # gan.train_on_batch expects input shape (batch_size, seq_len, 1)
    g_loss = gan.train_on_batch(real_data, real_labels)

    print(f"Epoch {epoch+1}/{epochs} | D Loss: {d_loss:.4f} | G Loss: {g_loss:.4f}")

# Predict next-day stock price
# Ensure the input to predict is also (1, seq_len, 1)
# X[-1] has shape (seq_len,). Reshape to (1, seq_len, 1)
input_for_prediction = X[-1].reshape(1, seq_len, 1)
next_day_prediction = generator.predict(input_for_prediction)

# The output shape of generator.predict is (1, seq_len, 1).
# Depending on what you want to predict (e.g., the next single value or the next sequence),
# you'll need to adjust how you extract the result.
# If predicting the next single price based on the last sequence:
# The generator output is a sequence of 'seq_len' values. If the goal is to predict the single next value,
# the generator architecture might need adjustment (e.g., a Dense(1) layer at the end if not predicting a sequence).
# Assuming the intention was to predict the *next sequence* of length seq_len starting from the last data point:

# If you truly meant the single next day price, you might need to reconsider the generator's output layer.
# For this code as written, it predicts a sequence of length seq_len.
# Round values and format output
# Convert array to list format with better readability
# Convert array to list format with better readability
formatted_prediction = next_day_prediction[0].tolist()

# Print in a compact format
print("Predicted Next-Day Prices Sequence:", formatted_prediction)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 406ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 1/15 | D Loss: 0.6746 | G Loss: 0.6911
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Epoch 2/15 | D Loss: 0.6792 | G Loss: 0.6908
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Epoch 3/15 | D Loss: 0.6800 | G Loss: 0.6904
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Epoch 4/15 | D Loss: 0.6806 | G Loss: 0.6900
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Epoch 5/15 | D Loss: 0.6811 | G Loss: 0.6897
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Epoch 6/15 | D Loss: 0.6815 | G Loss: 0.6894
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Epoch 7/15 | D Loss: 0.6817 | G Loss: 0.6890
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Epoch 8/15 | D Loss: 0.6818 | G Loss: 0.6885
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Epoch 9/15 | D Loss: 0.6820 | G Loss: 0.6881
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Epoch 10/15 | D Loss: 0.6823 | G Loss: 0.6877
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Epoch 11/15 | D Loss: 0.6825 | G Loss: 0.6873
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Epoch 12/15 | D Loss: 0.6827 | G Loss: 0.6869
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
E